See reddit.py

In [2]:
from psaw import PushshiftAPI
import pandas as pd

api = PushshiftAPI()

In [4]:
# Load white supremacist dataset to count posts over time

path = '../tmp/white_supremacist_corpus.pkl'
ws_data = pd.read_pickle(path)
print(len(ws_data))
ws_data.columns

4829033


Index(['text', 'timestamp', 'dataset', 'source', 'domain'], dtype='object')

In [8]:
# Select forum data, Group by year
# bymonth = ws_data.query('domain=="forum"').groupby(by=[ws_data.timestamp.dt.year, ws_data.timestamp.dt.month])['text'].count()
yearly = ws_data.query('domain=="forum"').groupby(by=ws_data.timestamp.dt.year)['text'].count()
yearly

timestamp
2001.0       1439
2002.0      13302
2003.0      27470
2004.0      50783
2005.0      54377
2006.0      80554
2007.0      63544
2008.0      74203
2009.0      94895
2010.0      62649
2011.0      56932
2012.0      87025
2013.0      71877
2014.0      63350
2015.0      71974
2016.0     110085
2017.0    1625386
2018.0    1367449
2019.0     656825
Name: text, dtype: int64

In [27]:
lookup = pd.DataFrame(yearly)
lookup['begin'] = pd.to_datetime(yearly.index.astype(int).astype(str), format='%Y')
lookup['end'] = lookup.begin.shift(-1, fill_value = datetime.datetime(2020,1,1))
lookup.index.name = 'year'
lookup.index = lookup.index.astype(int)
lookup.rename(columns={'text': 'post_count'}, inplace=True)
lookup

,post_count,begin,end
year,,,
2001,1439,2001-01-01,2002-01-01
2002,13302,2002-01-01,2003-01-01
2003,27470,2003-01-01,2004-01-01
2004,50783,2004-01-01,2005-01-01
2005,54377,2005-01-01,2006-01-01
2006,80554,2006-01-01,2007-01-01
2007,63544,2007-01-01,2008-01-01
2008,74203,2008-01-01,2009-01-01
2009,94895,2009-01-01,2010-01-01


In [37]:
import logging

handler = logging.StreamHandler()
handler.setLevel(logging.INFO)

logger = logging.getLogger('psaw')
# logger.setLevel(logging.INFO)
logger.setLevel(logging.ERROR)
logger.addHandler(handler)

In [70]:
# Scrape r/politics
import datetime
from tqdm.notebook import tqdm
import pdb

dfs = []
post_filter_list = ['id', 'selftext', 'title', 'author', 'created_utc', 'num_comments', 'score', 'brand_safe', 'over_18', 'domain', 'url', 'permalink']
comment_filter_list = ['id', 'parent_id', 'body', 'author', 'created_utc', 'score', 'permalink']
debug_limit = 100
real_limit = int(row.post_count * .7)

for index, row in tqdm(lookup.iterrows(), total=len(lookup)):
    print(index)
    posts = [post.d_ for post in api.search_submissions(subreddit='politics', after=row.begin, before=row.end, filter=post_filter_list, limit=debug_limit)]
    comments = [comment.d_ for comment in api.search_comments(subreddit='politics', after=row.begin, before=row.end, filter=comment_filter_list, limit=debug_limit)]
    print(len(comments))
    # Remove deleted posts
    if len(posts) == 0 and len(comments) == 0:
        print('\tskip')
        continue
    posts_df = pd.DataFrame([el for el in posts if el['selftext'] not in ['[deleted]', '[removed]']]).assign(post_type='submission')
    comments_df = pd.DataFrame([el for el in comments if el['body'] != '[deleted]']).assign(post_type='comment')
    # comments_df = pd.DataFrame(comments).assign(post_type='comment')
    print(f'\tRemoved {len(posts)-len(posts_df)} deleted posts')
    print(f'\tRemoved {len(comments)-len(comments_df)} deleted comments')
    # content = [el for el in posts+comments if el['selftext'] != '[deleted']
    # dfs.append(pd.DataFrame(content).sample(min(len(content), row.post_count)))
    dfs.append(pd.concat([posts_df, comments_df]).sample(min(len(posts_df)+len(comments_df), row.post_count)))
len(dfs)

  0%|          | 0/19 [00:00<?, ?it/s]

2001


/mnt/cephfs/cluster/carley/carley02/mamille3/white_supremacist_lang/conda_env/lib/python3.10/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


0
	skip
2002
0
	skip
2003
0
	skip
2004
0
	skip
2005
0
	skip
2006
0
	skip
2007
0
	Removed 46 deleted posts
	Removed 0 deleted comments
2008


/mnt/cephfs/cluster/carley/carley02/mamille3/white_supremacist_lang/conda_env/lib/python3.10/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


...

/mnt/cephfs/cluster/carley/carley02/mamille3/white_supremacist_lang/conda_env/lib/python3.10/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


100
	Removed 0 deleted posts
	Removed 7 deleted comments
2019


/mnt/cephfs/cluster/carley/carley02/mamille3/white_supremacist_lang/conda_env/lib/python3.10/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


100
	Removed 4 deleted posts
	Removed 0 deleted comments


13

In [108]:
data = pd.concat(dfs).reset_index(drop=True)
print(len(data))
data.head()

1875


,author,brand_safe,created_utc,domain,id,num_comments,over_18,permalink,score,selftext,title,url,created,post_type,body,parent_id
0,diogames,True,1.199129e+09,fallbackbelmont.blogspot.com,6487j,5.0,False,/r/politics/comments/6487j/the_next_the_time_t...,0.0,,The next the time the world needs saving they ...,http://fallbackbelmont.blogspot.com/2007/12/no...,1.199144e+09,submission,NaN,NaN
1,babblingpoet,True,1.199140e+09,andrewsullivan.theatlantic.com,648jx,0.0,False,/r/politics/comments/648jx/obama_like_so_many_...,0.0,,"""Obama, like so many of us, is conservative by...",http://andrewsullivan.theatlantic.com/the_dail...,1.199154e+09,submission,NaN,NaN
2,inquirer,True,1.199144e+09,politicalinquirer.com,648n0,1.0,False,/r/politics/comments/648n0/iowa_predictions_fo...,5.0,,Iowa: Predictions for Democrats,http://politicalinquirer.com/2007/12/31/fooser...,1.199158e+09,submission,NaN,NaN
3,inquirer,True,1.199143e+09,politicalinquirer.com,648mk,1.0,False,/r/politics/comments/648mk/the_use_of_fear_in_...,19.0,,The Use of Fear in American Politics: From the...,http://politicalinquirer.com/2007/12/29/the-bo...,1.199158e+09,submission,NaN,NaN
4,MikaelMN,True,1.199129e+09,opednews.com,6486f,1.0,False,/r/politics/comments/6486f/who_killed_the_kenn...,0.0,,Who Killed the Kennedys After All?,http://www.opednews.com/articles/opedne_mikael...,1.199143e+09,submission,NaN,NaN


In [110]:
import numpy as np

# data['text'] = data['title'].fillna('') + data.selftext.replace('', np.nan).fillna(data.body)
data['text'] = data['title'].fillna('') + data.selftext.fillna('') + data.body.fillna('')
data['text'].replace('', np.nan).count()

1875

In [111]:
data.text.sample(20)

524     House of Representatives will not vote on Mond...
1832    God your a moron, Joe. Of all the things you C...
113                           Hamas Legalizes Crucifixion
1294    I listed Fox as part of it too.  Fox just offe...
790     A Free Trade New Year's Resolution For Congres...
767                                       and work where?
983     CNN Anchor Shocked To Learn There Are 14 Women...
1155    Istanbul nightclub attack: Several injured aft...
151        Did Karl Rove Leave His Wife For Jeff Gannon? 
251     A True Hero: Staff Sergeant Salvatore Giunta. ...
59      It is now 2009 in Gaza: A new year ushered in ...
283     Sara Benincasa, on the GOP spending money at H...
1315    Bernie Sanders Warns Donald Trump Against Priv...
27      Hillary Clinton, Sinbad, and Cheryl Crow: risk...
998     The narrative is important because it serves a...
845     The [Mercantilism](http://en.wikipedia.org/wik...
1220    Honest question, what are the downsides of sel...
770           

In [99]:
data.title.fillna('').count()

1875

In [112]:
data[data.eq('[removed]').any(1)]

,author,brand_safe,created_utc,domain,id,num_comments,over_18,permalink,score,selftext,title,url,created,post_type,body,parent_id,text
188,clembo,True,1.262302e+09,self.politics,akdxe,0.0,False,/r/politics/comments/akdxe/why_does_the_front_...,2.0,[removed],Why does the front page of politics always con...,https://www.reddit.com/r/politics/comments/akd...,1.262317e+09,submission,NaN,NaN,Why does the front page of politics always con...
195,newmediajournal,True,1.262289e+09,self.politics,akcdt,0.0,False,/r/politics/comments/akcdt/tsa_subpoenas_blogg...,1.0,[removed],"TSA Subpoenas Bloggers, Demands Names of Sources",https://www.reddit.com/r/politics/comments/akc...,1.262303e+09,submission,NaN,NaN,"TSA Subpoenas Bloggers, Demands Names of Sourc..."
233,DangerMouseUS,True,1.293833e+09,self.politics,eubwl,0.0,False,/r/politics/comments/eubwl/a_pretty_interestin...,2.0,[removed],A pretty interesting quote from Herman Göring ...,https://www.reddit.com/r/politics/comments/eub...,1.293848e+09,submission,NaN,NaN,A pretty interesting quote from Herman Göring ...
264,mschase1972,True,1.293832e+09,self.politics,eubme,0.0,False,/r/politics/comments/eubme/2010_success_and_fa...,1.0,[removed],2010: Success And Failure Merge,https://www.reddit.com/r/politics/comments/eub...,1.293846e+09,submission,NaN,NaN,2010: Success And Failure Merge[removed]
1135,[deleted],NaN,1.451606e+09,NaN,cyhtlh4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.451621e+09,comment,[removed],t1_cyhtjbq,[removed]
1184,[deleted],NaN,1.483229e+09,NaN,dbumhol,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.483243e+09,comment,[removed],t1_dbu396e,[removed]
1213,[deleted],NaN,1.483229e+09,NaN,dbuml2z,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.483243e+09,comment,[removed],t1_dbuivvv,[removed]
1214,[deleted],NaN,1.483229e+09,NaN,dbumm0t,NaN,NaN,NaN,-16.0,NaN,NaN,NaN,1.483243e+09,comment,[removed],t1_dbumc4f,[removed]
1250,[deleted],NaN,1.483229e+09,NaN,dbumknj,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.483243e+09,comment,[removed],t1_dbulfxy,[removed]
1260,[deleted],NaN,1.483229e+09,NaN,dbumiar,NaN,NaN,NaN,-4.0,NaN,NaN,NaN,1.483243e+09,comment,[removed],t1_dbuim87,[removed]


In [80]:
with_text = data[data['selftext'].map(lambda x: not isinstance(x, float) and len(x) > 0)]
with_text['selftext'].sample(10)

38    They must use their resources and influence to...
62                         See http://www.xkcd.com/834/
21    Hello world, this is my first post here on Red...
68                 I'm sick of this Obama, good bye. \n
10    Rather than be completely turned off with Redd...
69    If I could draw, I would draw a political cart...
17    Are there any reasonable third-party/independe...
36    The Defense Authorization act was signed by Ob...
78                                            [removed]
4     If only people have the right to free speech, ...
Name: selftext, dtype: object

In [28]:
posts = [post.d_ for post in posts]
posts

[{'author': '[deleted]',
  'author_flair_background_color': '',
  'author_flair_css_class': None,
  'author_flair_text': None,
  'author_flair_text_color': 'dark',
  'brand_safe': True,
  'contest_mode': False,
  'created_utc': 1262303983,
  'domain': 'apnews.myway.com',
  'full_link': 'https://www.reddit.com/r/politics/comments/ake3n/im_ok_with_rush_limbaugh_but_im_not_thrilled_that/',
  'gilded': 0,
  'id': 'ake3n',
  'is_crosspostable': False,
  'is_reddit_media_domain': False,
  'is_self': False,
  'is_video': False,
  'link_flair_richtext': [],
  'link_flair_text_color': 'dark',
  'link_flair_type': 'text',
  'locked': False,
  'media_embed': {},
  'no_follow': True,
  'num_comments': 0,
  'num_crossposts': 0,
  'over_18': False,
  'parent_whitelist_status': 'all_ads',
  'permalink': '/r/politics/comments/ake3n/im_ok_with_rush_limbaugh_but_im_not_thrilled_that/',
  'retrieved_on': 1522880397,
  'rte_mode': 'markdown',
  'score': 1,
  'secure_media_embed': {},
  'selftext': '[delet